<a href="https://colab.research.google.com/github/sokrypton/ColabDesign/blob/v1.1.0/mpnn/examples/proteinmpnn_in_jax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ProteinMPNN in Jax!

In [ ]:
#@title import libraries
import warnings, os
warnings.simplefilter(action='ignore', category=FutureWarning)
try:
  import colabdesign
except:
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git@v1.1.0")
  os.system("ln -s /usr/local/lib/python3.7/dist-packages/colabdesign colabdesign")

from colabdesign.af import mk_af_model, clear_mem
from colabdesign.af.alphafold.common import residue_constants
from colabdesign.af.prep import order_aa
from colabdesign.mpnn import mk_mpnn_model
from IPython.display import HTML

import numpy as np
import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp

from google.colab import files

def get_pdb(pdb_code=""):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  else:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    return f"{pdb_code}.pdb"

def af2mpnn(self):
  atom_idx = tuple(residue_constants.atom_order[k] for k in ["N","CA","C","O"])
  X = self._inputs["batch"]["all_atom_positions"][:,atom_idx]
  mask = self._inputs["batch"]["all_atom_mask"][:,1]
  inputs ={"X":X,
           "S":self._wt_aatype,
           "mask":mask,
           "residue_idx":self._inputs["residue_index"],
           "chain_idx":self._inputs["asym_id"],
           "key":self.key(),
           "bias":self._inputs["bias"]}
  
  if "fix_pos" in self.opt:
    i,p = np.arange(X.shape[0]), self.opt["fix_pos"]
    inputs["bias"][p] += 1e8 * np.eye(20)[self._wt_aatype[p]]
    inputs["decoding_order"] = np.append(np.take(i,p),np.random.permutation(np.delete(i,p)))
    inputs["fix_pos"] = self.opt["fix_pos"]
  return inputs

def seq2aa(seq):
  return "".join([order_aa[aa] for aa in seq])

In [ ]:
import re
#@markdown ### ProteinMPNN options
model_name = "v_48_020" #@param ["v_48_002", "v_48_010", "v_48_020", "v_48_030"]
#@markdown ### Input Options
pdb='6MRR' #@param {type:"string"}
pdb_path = get_pdb(pdb)
#@markdown - leave blank to get an upload prompt
chains = "A" #@param {type:"string"}
chains = re.sub("[^A-Za-z]+",",", chains)
fix_pos = "" #@param {type:"string"}
if fix_pos == "": fix_pos = None
#@markdown - specify which positions to keep fixed in the sequence (example: `1,2-10`)
#@markdown - you can also specify multichain constrains (example: `A1-10,B1-20`)

clear_mem()
af_model = mk_af_model(protocol="fixbb", use_alphafold=False)
af_model.prep_inputs(pdb_filename=pdb_path, chain=chains, fix_pos=fix_pos)
mpnn_model = mk_mpnn_model(model_name)
print("length",sum(af_model._lengths))

In [ ]:
%%time
from scipy.special import log_softmax
#@markdown ### Design Options
num_seqs = 32 #@param ["0","1", "2", "4", "8", "16", "32", "64", "128", "256"] {type:"raw"}

sampling_temp = 0.1 #@param ["0.0001", "0.1", "0.15", "0.2", "0.25", "0.3", "0.5", "1.0"] {type:"raw"}
#@markdown - Sampling temperature for amino acids, T=0.0 means taking argmax, T>>1.0 means sample randomly.
rm_aa = "" #@param {type:"string"}
rm_aa = ",".join(list(re.sub("[^A-Z]+","",rm_aa.upper())))
if rm_aa == "": rm_aa = None
#@markdown - specify amino acid(s) to exclude (example: `C,A,T`)
score_wildtype = True #@param {type:"boolean"}

af_model.set_seq(rm_aa=rm_aa)
seqs = []
for n in range(num_seqs + score_wildtype):
  inputs = af2mpnn(af_model)
  if score_wildtype and n == 0:
    outputs = mpnn_model.score(**inputs)
  else:
    outputs = mpnn_model.sample(**inputs)
  outputs = jax.tree_map(np.array, outputs)

  mask = inputs["mask"].copy()
  if "fix_pos" in inputs:
    mask[inputs["fix_pos"]] = 0

  score = -(outputs["seq"] * log_softmax(outputs["logits"],-1)).sum(-1)
  score = (score * mask).sum() / mask.sum()

  seq = outputs["seq"].argmax(-1)
  seqid = seq == af_model._wt_aatype
  seqid = (seqid * mask).sum() / mask.sum()
  
  print(f'>score:{score:.3f}_seqid:{seqid:.3f}\n{seq2aa(seq)}')

In [ ]:
#@markdown ### amino acid probabilties (unconditional)
import plotly.express as px
inputs = af2mpnn(af_model)
inputs.pop("S") # remove sequence
pssm = jax.nn.softmax(mpnn_model.score(**inputs)["logits"])
fig = px.imshow(np.array(pssm).T,
               labels=dict(x="positions", y="amino acids", color="probability"),
               y=residue_constants.restypes,
               zmin=0,
               zmax=1,
               template="simple_white",
              )
fig.update_xaxes(side="top")
fig.show()